In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import pandas as pd
import opt_tugba
import sys
import os
import warnings
warnings.filterwarnings("ignore")
import cvxpy

import matplotlib 

plt.rcParams['figure.constrained_layout.use'] = True
matplotlib.rc('xtick', labelsize=10) 
matplotlib.rc('ytick', labelsize=10) 
matplotlib.rcParams.update({'font.size': 15})
cvxpy.installed_solvers()

['ECOS', 'ECOS_BB', 'GUROBI', 'OSQP', 'SCS']

In [2]:
#  initialize the problem
new_event = True 
interval_size_hours = 15 / 60 

# 5 minute intervals
tariff = np.ones((int(24 / interval_size_hours),)) * 10
tariff[32:40] = np.ones(shape = (8,)) * 15
tariff[60:80] = np.ones(shape = (20,)) * 20
pow_max = 6.6
pow_min = 0
overstay_duration= 1



In [66]:
df_flex = pd.DataFrame(index =  np.arange(6,96,6), columns = np.arange(5,55,5))
df_asap = pd.DataFrame(index =  np.arange(6,96,6), columns = np.arange(5,55,5))

268

In [3]:
%%time
count = 0 
for arrival_interval in [72, 96, 108]: # 5 min interval
    for duration in [12, 30, 48, 96]: # 5 min interval 
        for e_needed in np.arange(5,25,5):
            for z0 in [np.array([1,1,1,1]) , np.array([100,100,1,1]), np.array([150,150,1,1]), np.array([100,200,1,1])]:
                for v0 in [np.array([0.33, 0.33, 0.33]), np.array([0.50, 0.50, 0.0]), np.array([0.25, 0.50, 0.25])]:
                    if duration < int(np.ceil((e_needed / 6.6) * 12)):
                        print("Not enough time")
                    else:

                        event = {"time" : arrival_interval, 
                                "pow_min" : pow_min, 
                                "pow_max" : pow_max, 
                                "overstay_duration" : overstay_duration, 
                                "duration" : duration, 
                                "e_need":e_needed}
                        
                        par = opt_tugba.Parameters(TOU = tariff, 
                                                    z0 = z0.reshape(4,1) ,
                                                    v0 = v0.reshape(3,1) , 
                                                    eff = 1.0, lam_x = 0, lam_h_c= 0, lam_h_uc=0,  lam_z_uc = 0,  lam_z_c = 0, Ts = interval_size_hours) 
                        
                        prb = opt_tugba.Problem(par = par, event = event)
                        
                        opt = opt_tugba.Optimization(par, prb)
                        
                        res = opt.run_opt()

                        # df_flex.loc[duration,e_needed] = res['tariff_flex'] * 6.6 ## convert to cents / hour
                        # df_asap.loc[duration,e_needed] = res['tariff_asap'] * 6.6 ## convert to cents / hour

                        record_results(event, par, prb, opt, res, count)
                        count +=1


            



TypeError: __init__() got an unexpected keyword argument 'mu'

In [34]:
def record_results(event, par, prb, opt, res, test):
    days = 0 
    num_interval= len(par.TOU)
    interval_size = par.Ts

    start_hour = int(res["time_start"] * par.Ts // 1)
    start_minute = int(np.ceil(60 * ( res['time_start'] *par.Ts % 1)))
    flex_end_hour = int(res["time_end_flex"] * par.Ts // 1)
    flex_end_minute= int(np.ceil(60 * ( res['time_end_flex'] *par.Ts % 1)))
    asap_end_hour = int(res["time_end_asap"] * par.Ts // 1)
    asap_end_minute = int(np.ceil(60 * ( res['time_end_asap'] *par.Ts % 1)))

    # 5 min interval
    timerange = pd.date_range("1/1/2021", periods=num_interval, freq="{}min".format(5))
    ## all time frame 
    TOU_cost = pd.Series(par.TOU, index = timerange, name = "TOU Cost ($)")


    # Event parameters: arrival_time, soc_init, soc_need, departure
    opt_time_start  = pd.Timestamp(year=2021, month=1, day=1, hour = start_hour, minute = start_minute)
    opt_time_end_flex  =pd.Timestamp(year=2021, month=1, day=1, hour = flex_end_hour,  minute = flex_end_minute)
    opt_time_end_asap  = pd.Timestamp(year=2021, month=1, day=1, hour = asap_end_hour, minute = asap_end_minute)

    flex_ts = pd.date_range(start=opt_time_start , end=opt_time_end_flex, freq="{}H".format(par.Ts))
    asap_ts = pd.date_range(start=opt_time_start , end=opt_time_end_asap, freq="{}H".format(par.Ts))

    # Optimization outputs 

    ## Question: Why flex output length mismatches the timelength? 
    # print(opt.opt_flex_powers.shape)
    opt_flex_powers = pd.Series(res['flex_powers'].reshape(len(res['flex_powers']),), index = flex_ts[:-1], name = "Flew Power (kW)")
    opt_asap_powers = pd.Series(res['asap_powers'].reshape(len(res['asap_powers']),), index = asap_ts[:-1], name = "ASAP Power (kW)")
    opt_flex_energy = pd.Series(res['flex_e_delivered'].reshape(len(res['flex_e_delivered']),), index = flex_ts, name = "Energy Delivered (kWh)")
    # event = prb.event
    # print(event)

    ## PLOT FOR 2 days 
    days =2 
    ts = pd.date_range(timerange[0], periods=num_interval * days, freq="{}H".format(interval_size ))
    df_plot = pd.DataFrame(index = ts, 
                            data = {"TOU Cost (cents/hour)": list(TOU_cost.values* 6.6) * days, 
                            "Flex Power (kW)": opt_flex_powers, 
                            "ASAP Power (kW)":opt_asap_powers, 
                            "Energy Delivered, Flex (kWh)":opt_flex_energy})

    df_plot["Energy Delivered, Flex (kWh)"] = df_plot["Energy Delivered, Flex (kWh)"].fillna(method='bfill').fillna(method='ffill')
    df_plot = df_plot.fillna(0)
    df_plot['Station Max Power (kW)'] = event['pow_max']
    df_plot['Station Min Power (kW)'] = event['pow_min']
    # df_plot['Arrival SOC'] = event['SOC_init']
    # df_plot['Departure SOC Demand'] = event['SOC_need']
    df_plot['Arrival Time'] = event['time']
    df_plot['Duration'] = event['duration']



    plot_power_energy(df_plot, prb, test, start_hour,start_minute,flex_end_hour ,flex_end_minute)
    plot_prices_and_probs(res, test)
    save_the_data(df_plot, res, test)




In [25]:


def plot_arrival_departure(ax_list, start_hour,start_minute,flex_end_hour ,flex_end_minute,event):
    for ax in ax_list:
        arrival_time = pd.Timestamp(year=2021, month=1, day=1, hour = start_hour, minute = start_minute)
        departure_time = pd.Timestamp(year=2021, month=1, day=1, hour = flex_end_hour, minute = flex_end_minute)
        ax.axvspan(arrival_time, departure_time, facecolor='b', alpha=0.25)

def plot_max_min_power(ax_list, df_plot):
    for ax in ax_list:
        ax.plot(df_plot['Station Max Power (kW)'], label="Station Max Power (kW", color = 'red',linestyle='--')
        # ax.plot(df_plot['Station Min Power (kW)'],label="Station Max Power (kW")))

def x_axis_labels(ax_list, cuts=[0,6,12,18]):
    import matplotlib.dates as mdates
    hlocator = mdates.HourLocator(byhour=cuts, interval=1, tz=None)
    for ax in ax_list:
        # ax.xaxis.set_major_locator(loc)
        ax.xaxis.set_major_locator(hlocator)
        ax.xaxis.set_major_formatter(mdates.DateFormatter("%H"))

def add_legend(ax_list):
    for ax in ax_list:
        ax.legend()

In [22]:
def plot_power_energy(df_plot, prb, test, start_hour,start_minute,flex_end_hour ,flex_end_minute):
    plt.rcParams['figure.constrained_layout.use'] = True

    fig,(ax1, ax2, ax3, ax4) = plt.subplots(4,1,figsize=(12,9) ,)

    x_lim  = (list(df_plot.index)[0],list(df_plot.index)[-1])
    ax1.plot(df_plot['TOU Cost (cents/hour)'])

    ax1.set_xlim(x_lim)
    ax1.grid()
    ax1.set_xlabel("Time")
    ax1.set_ylabel("TOU (cents / hour)")


    ax2.plot(df_plot["Flex Power (kW)"], label = "Power Delivered (kW)")
    ax2.set_xlim(x_lim)
    ax2.grid()
    ax2.set_xlabel("Time")
    ax2.set_ylabel("Power (kW)")
    ax2.set_title("Flex Charging Optimal Power")

    ax3.plot(df_plot["ASAP Power (kW)"], label = "Power Delivered (kW)")
    ax3.set_xlim(ax1.set_xlim(x_lim))
    ax3.grid()
    ax3.set_xlabel("Time")
    ax3.set_ylabel("Power (kW)")
    ax3.set_title("ASAP Charging Optimal Power")

    ax4.plot(df_plot["Energy Delivered, Flex (kWh)"], label  = 'Energy Delivered (kWh)')
    ax4.axhline(prb.e_need,  linestyle='--',color='red', label = 'Energy Demand (kWh)')
    ax4.set_xlim(x_lim)
    ax4.grid()
    ax4.set_xlabel("Time")
    ax4.set_ylabel("Energy Level (kWh)")

    ax4.set_title("Flex Charging Energy Delivered (kWh)")

    plot_arrival_departure([ax2,ax3], start_hour,start_minute,flex_end_hour ,flex_end_minute,event)
    plot_max_min_power([ax2,ax3], df_plot)
    x_axis_labels([ax1,ax2,ax3,ax4], cuts=[0,5,10,15,20])
    add_legend([ax1,ax2,ax3,ax4])
    # fig.suptitle("{} , Solver=Gurobi".format(objective))
    plt.savefig(os.path.join("Figures/", "test_full_code_{}.pdf".format(test)),format='pdf')

    plt.show()
    plt.close()

In [39]:

def save_the_data(df_plot, res, test):
    from openpyxl import load_workbook
    path = "parameters_21feb.xlsx"
    book = load_workbook(path)
    writer = pd.ExcelWriter(path, engine = 'openpyxl')
    writer.book = book

    df_plot['tariff_flex (cents/hour)'] = res['tariff_flex'] * 6.6

    df_plot['tariff_asap (cents/hour)'] = res['tariff_asap'] * 6.6
    
    df_plot['z0'] = par.z0
    df_plot['v0'] = par.v0

    df_plot['prob_asap'] = res['prob_asap']
    df_plot['prob_flex'] = res['prob_flex']
    df_plot['prob_leave'] = res['prob_leave']
    df_plot['e_need'] = res['e_need']
    
    df_plot.to_excel(writer,sheet_name="test_{}".format(test))

    writer.save()
    writer.close()


dict_keys(['z', 'tariff_flex', 'tariff_asap', 'tariff_overstay', 'peak_pow', 'flex_e_delivered', 'flex_powers', 'asap_powers', 'v', 'prob_flex', 'prob_asap', 'prob_leave', 'J', 'J_sub', 'z_iter', 'v_iter', 'rev_flex', 'rev_asap', 'num_iter', 'prb', 'par', 'time_start', 'time_end_flex', 'time_end_asap'])

In [16]:
# PLOT FOR EACH ITERATION
def plot_prices_and_probs(res, test):
    fig,(axes) = plt.subplots(5,1,figsize=(12,12))
    # Objective Value for each iteration 
    ax = axes[0]
    ax.plot(res['J'], label = 'Total Objective Value')
    ax.plot(res['J_sub'][0],label='Flex Charging Objective')
    ax.plot(res['J_sub'][1],label='Asap Charging Objective')
    ax.plot(res['J_sub'][2],label='Leave Objective')
    ax.margins(0)
    ax.grid()
    ax.set_xlabel("Iteration")
    ax.set_ylabel("$J(x,z,v)$")
    ax.set_title("Objective Value")
    ax.legend()

    # Flex Charging Revenue for each iteration and flex charging costs 
    ax = axes[1]
    ax.plot(res['rev_flex'],linewidth = 2)
    ax.set_xlim(0,res['num_iter'] - 1)
    ax.grid()
    ax.set_xlabel("Iteration")
    ax.set_ylabel("cents")
    ax.set_title("Flex Charging Revenue")

    ax_twin  = ax.twinx()
    ax_twin.plot(res['z_iter'][0,:] * 6.6,linestyle = "--",color ="red",linewidth = 2)
    ax_twin.set_ylabel("cents / hour", color = 'red')


    # Asap Charging Revenue for each iteration and asap charging costs 
    ax = axes[2]
    ax.plot(res['rev_asap'],linewidth = 2)
    ax.set_xlim(0,res['num_iter'] - 1)
    ax.grid()
    ax.set_xlabel("Iteration")
    ax.set_ylabel("cents")
    ax.set_title("ASAP Charging Revenue")

    ax_twin  = ax.twinx()
    ax_twin.plot(res['z_iter'][1,:] * 6.6,linestyle = "--",color ="red",linewidth = 2)
    ax_twin.set_ylabel("cents / hour", color = 'red')

    # Probability of Charging Flex, Asap and Leaving at each iteration 
    ax=axes[3]
    ax.bar(x = np.arange(res['num_iter']),height = res['v_iter'][0,:], label = 'Flex Charging')
    ax.bar(x = np.arange(res['num_iter']),height = res['v_iter'][1,:], label = 'ASAP Charging', bottom = res['v_iter'][0,:])
    ax.bar(x = np.arange(res['num_iter']),height = res['v_iter'][2,:], label = 'Leave', bottom = res['v_iter'][1,:] + res['v_iter'][0,:])
    ax.legend()
    ax.grid()
    ax.set_xlabel("Iteration")
    ax.set_ylabel("$P(i)$")
    ax.set_title("Probability")

    # Price of each option maybe 
    ax=axes[4]
    ax.bar(x = np.arange(res['num_iter']),height = res['z_iter'][0,:] * 6.6, label = 'Flex Charging')
    ax.bar(x = np.arange(res['num_iter']),height = res['z_iter'][1,:] * 6.6, label = 'ASAP Charging', bottom = res['z_iter'][0,:]* 6.6)
    # ax.bar(x = np.arange(res['num_iter']),height = res['z_iter'][2,:], label = 'Overstay', bottom = res['z_iter'][0,:] + res['z_iter'][1,:])
    # ax.bar(x = np.arange(res['num_iter']),height = res['z_iter'][3,:], label = 'Leave',bottom = (res['z_iter'][0,:] + res['z_iter'][1,:] + res['z_iter'][2,:]))
    ax.legend()
    ax.grid()
    ax.set_xlabel("Iteration")
    ax.set_title("Price")
    ax.set_ylabel("cents / hour", color = 'red')
    plt.savefig(os.path.join("Figures/", "power_price_{}.pdf".format(test)),format='pdf')

    # 

In [149]:
## Wgy scale dependent? 
res['tariff_flex']

21.015485386001675

In [150]:
        ### Read parameters 
        N_flex = prb.N_flex
        N_asap = prb.N_asap
        TOU = prb.TOU
        station_pow_max = prb.station_pow_max
        lam_x = par.lam_x
        lam_h_c = par.lam_h_c
        lam_h_uc = par.lam_h_uc
        user_SOC_init  =  prb.user_SOC_init
        user_SOC_need = prb.user_SOC_need
        delta_k = par.Ts
        eff = par.eff
        user_bat_cap = prb.user_batt_cap  
        print("Lam_X:",lam_x)
        print(N_flex, N_asap, lam_x, user_SOC_init, user_SOC_need,eff)
        objective = "J = u.T @ TOU + norm2 * {} + lam_h_c * 1 / z[2] \n + cp.sum(station_pow_max * (TOU[:N_asap] - z[1])) \n v[2] * cp.sum(TOU[:N_asap])".format(lam_x)

AttributeError: 'Problem' object has no attribute 'user_SOC_init'

In [ ]:
def calculate_probs(x1,x2):
    u_reg =  0.3411 -0.0184*(x1 - x2)*.5
    u_sch = 0.0184*(x1 - x2)*.5
    u_leave = -1. + 0.005*(np.mean([x1, x2]))

    denom = np.exp(u_reg) + np.exp(u_sch) + np.exp(u_leave)

    p_reg = np.exp(u_reg) / denom
    p_sch = np.exp(u_sch) / denom
    p_leave = np.exp(u_leave) / denom 
    print("Regular:",np.round(p_reg,3)," Scheduled:",np.round(p_sch,3),"Leave:",np.round(p_leave,3))


x1 = 150
x2 = 150

calculate_probs(x1,x2)


Regular: 0.442  Scheduled: 0.314 Leave: 0.244
